In [18]:
import numpy as np
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression
import pandas as pd #for reading csv files
from sklearn.model_selection import GridSearchCV  # Import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

In [19]:
training=pd.read_csv('training_data.csv', sep=',')
test=pd.read_csv('songs_to_classify.csv', sep=',')

only use five features version

In [20]:
# select which features to use
features = ['danceability','key','loudness','instrumentalness','liveness']

# i learn how to do one-hot encode from chatgpt because i realized there is no size meaning in the key feature.  
encoder = OneHotEncoder(sparse=False)

X_train_selected = training[features]
X_test_selected = test[features]
# OneHotEncode the 'key' feature
X_train_key_encoded = encoder.fit_transform(X_train_selected[['key']])
X_test_key_encoded = encoder.transform(X_test_selected[['key']])

X_train_no_key = X_train_selected.drop('key', axis=1).values
X_test_no_key = X_test_selected.drop('key', axis=1).values

X_train = np.hstack([X_train_no_key, X_train_key_encoded])
X_test = np.hstack([X_test_no_key, X_test_key_encoded])

# extract labels
y_train = training.loc[:, 'label'].values

In [21]:
# Normalize data

X_trainn = X_train * 1 / np.max(np.abs(X_train), axis=0)
X_testn = X_test * 1 / np.max(np.abs(X_test), axis=0)

using all features version

In [22]:
# i learned how to choose all the features from chatgpt but now i think i could figure it out myself

#encoder = OneHotEncoder(sparse=False)
#X_train_key_encoded = encoder.fit_transform(X_train_selected[['key']])
#X_test_key_encoded = encoder.transform(X_test_selected[['key']])
#X_train_time_signature_encoded = encoder.fit_transform(training[['time_signature']])
#X_test_time_signature_encoded = encoder.transform(test[['time_signature']])

# 移除 'key' 和 'time_signature' 列，并保留其他特征
#X_train_no_key_time = training.loc[:, (training.columns != 'key') & (training.columns != 'time_signature')].drop('label', axis=1).values
#X_test_no_key_time = test.loc[:, (test.columns != 'key') & (test.columns != 'time_signature')].values

# 合并 OneHot 编码后的 'key' 和 'time_signature' 与其他特征
#X_train = np.hstack([X_train_no_key_time, X_train_key_encoded, X_train_time_signature_encoded])
#X_test = np.hstack([X_test_no_key_time, X_test_key_encoded, X_test_time_signature_encoded])
# 提取标签
#y_train = training.loc[:, 'label'].values

In [23]:
# Initialize the logistic regression model with L2 regularization
logistic_model = LogisticRegression(penalty='l2', solver='lbfgs', C=1.0)  # C is the inverse of regularization strength

In [24]:
# Define the parameter grid

# i use chatgpt to generate this code for choosing these hyperparameters using GridSearchCV. 
# I know this method but i was not sure how to implement it in code.
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # 正则化强度的不同值
    'solver': ['lbfgs', 'liblinear'],  # 不同的求解器
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=logistic_model, param_grid=param_grid, 
                           scoring='accuracy', cv=5)  # 5-fold cross-validation

# Train the model using GridSearchCV
grid_search.fit(X=X_trainn, y=y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best parameters: {best_params}')

Best parameters: {'C': 10, 'solver': 'lbfgs'}


In [25]:
# Make predictions with the best model

# Chagpt designed this output format for me and i asked it to remove spaces between elements so that i could use it for submission
best_model = grid_search.best_estimator_
predictions = best_model.predict(X=X_testn).reshape(-1, 1).astype(int).reshape(1, -1)
#print(predictions)
print(''.join(map(str, predictions.flatten())))

00010001001101001011001000110010111001011101110111101001100011000111100001000110111110111010001111110010000001110011110001101111101111111101101110001011101111110101100100111001001001101100100010011111


Cross Validation

In [26]:
# i use chatgpt to do cross validation because I need a standard to evaluate the pros and cons of different models
cv_scores_lr = cross_val_score(logistic_model, X_trainn, y_train, cv=5, scoring='accuracy')
print(f'Logistic Regression cross-validation accuracy: {cv_scores_lr.mean():.4f} ± {cv_scores_lr.std():.4f}')

Logistic Regression cross-validation accuracy: 0.7280 ± 0.0344
